In [ ]:
import pandas as pd
import numpy  as np
import random
import time
from rtsvg import *
rt = RACETrack()

In [ ]:
passage = "Apollo 13 (April 11–17, 1970) was the seventh crewed mission in the Apollo space program and the "              + \
          "third meant to land on the Moon. The craft was launched from Kennedy Space Center on April 11, 1970, "          + \
          "but the lunar landing was aborted after an oxygen tank in the service module (SM) ruptured two days "           + \
          "into the mission, disabling its electrical and life-support system. The crew, supported by backup systems "     + \
          "on the lunar module (LM), instead looped around the Moon in a circumlunar trajectory and returned safely "      + \
          "to Earth on April 17. The mission was commanded by Jim Lovell, with Jack Swigert as command module (CM) pilot " + \
          "and Fred Haise as lunar module (LM) pilot. Swigert was a late replacement for Ken Mattingly, who was "          + \
          "grounded after exposure to rubella."

def junk():
    _num_   = random.randint(100,5000)
    _words_ = []
    for i in range(_num_):
        _word_ = []
        for i in range(random.randint(3,8)): _word_.append(chr(random.randint(ord('a'), ord('z'))))
        _words_.append(''.join(_word_))
    return ' '.join(_words_)

passage_long = passage + junk() + passage + junk() + passage

entity_extractions = {}
# ChatGPT prompt:  "Extract the entities from the following passage.  Place the entities into a python list structure."
entity_extractions['chatgpt'] = ['Apollo 13','April 11–17, 1970',
 'seventh crewed mission','Apollo space program','Moon','Kennedy Space Center',
 'April 11, 1970','oxygen tank','service module (SM)','lunar landing','life-support system',
 'lunar module (LM)','circumlunar trajectory','Earth','April 17','Jim Lovell',
 'Jack Swigert','command module (CM) pilot','Fred Haise','Ken Mattingly','rubella']

# Claude Prompt: "Extract the entities from the following passage.  Place the entities into a python list structure."
entity_extractions['claude.ai'] = ["Apollo 13","Apollo space program","Moon",
"Kennedy Space Center","oxygen tank","service module","SM","lunar module","LM","Earth",
"Jim Lovell","Jack Swigert","command module","CM","Fred Haise","Ken Mattingly","rubella"]

# Gemini Prompt: "Extract the entities from the following passage.  Put the entities into a python list structure."
entity_extractions['gemini'] = entities = ["Apollo 13","Apollo space program","Kennedy Space Center",
"service module","lunar module","circumlunar trajectory","Jim Lovell","Jack Swigert","Fred Haise","Ken Mattingly","rubella"]

# Mistral 7B 3.1 Instruct Prompt: "Extract the entities from the following passage.  Put the entities into a python list structure."
entity_extractions['mistral_7b_31'] = ["Apollo 13","April 11","April 17, 1970","Apollo space program","Moon",
"Kennedy Space Center","April 11, 1970","lunar landing","service module (SM)","electrical and life-support system",
"lunar module (LM)","Jim Lovell","Jack Swigert","Fred Haise","command module (CM)","Ken Mattingly","rubella"]

# Llama 32 3b Prompt: "Extract the entities from the following passage.  Put the entities into a python list structure."
# ... system prompt is "You are a helpful AI assistant."
# ... note from the system was "Note that some of these entities may be classified as different types (e.g., "April" is a month, while "April 11, 1970" is a date), but I have treated them as single entities for the purpose of this exercise."
entity_extractions['llama32_3b'] = ["Apollo 13","April","1970","Apollo space program","Moon","Kennedy Space Center",
"service module (SM)","oxygen tank","electrical system","life-support system","lunar module (LM)","circumlunar trajectory","Earth",
"Jim Lovell","Jack Swigert","command module (CM)","Fred Haise","Ken Mattingly","rubella"]

entity_extractions['llama32_1b'] = ["Apollo 13", "April 11–17, 1970", "Kennedy Space Center", "Oxygen tank", "Service module",
"Lunar landing", "Lunar module", "Command module", "Commander", "Commander's seat",
"Mission", "Mission commander", "Mission pilot", "Backup systems", "Lunar module pilot",
"Lunar module commander", "Rubella", "Ken Mattingly", "Jim Lovell", "Jack Swigert", "Fred Haise"]

entity_extractions_lu = {}
for k,v in entity_extractions.items():
    _lu_ = {}
    for i in v: _lu_[i] = rt.co_mgr.getColor(i)
    entity_extractions_lu[k] = _lu_
_tb_     = rt.textBlock(passage, word_wrap=True)
_svg_    = _tb_.pixelRepr(lu=entity_extractions_lu['chatgpt'],   w=256)
w, h     = rt.__extractSVGWidthAndHeight__(_svg_)

_labels_ = []
_spacer_ = rt.spacer(20, 24, '#000000')
for _engine_ in entity_extractions_lu: _labels_.extend([rt.labeler([_engine_], w, 24), _spacer_])
_labels_svg_ = rt.tile(_labels_[:-1])
_spacer_ = rt.spacer(20, h, '#000000')
_pixreps_ = []
for _engine_ in entity_extractions_lu: _pixreps_.extend([_tb_.pixelRepr(lu=entity_extractions_lu[_engine_], w=256), _spacer_])
_pixreps_svg_ = rt.tile(_pixreps_[:-1])

rt.tile([_labels_svg_, _pixreps_svg_], horz=False)


In [3]:
#_tbl_ = rt.textBlock(passage_long, word_wrap=True)
#_tbl_

In [ ]:
#
# ... copied into the framework - 2024-10-05 14:00EST
#
def PROTOTYPE_labeler(text_tuples, w, h=None, txt_h=16, x_ins=2, y_ins=4, h_gap=2):
    '''
    labeler - create an svg label from a list of text tuples

    parameters
    ----------
    text_tuples - list of text tuples, examples are as follows
        ['this', 'is', 'a', 'test'] # simple strings for labels -- defaults for all other values
        - or -
        a tuple consisting of the following items...
        ('my string')
        ('my string', my_txt_h)
        ('my string', my_txt_h, my_hex_color)
        ('my_string', my_txt_h, my_hex_color, my_h_gap)
        ('my_string', my_txt_h, my_hex_color, my_h_gap, my_top_gap)
        ('my_string', my_txt_h, my_hex_color, my_h_gap, my_top_gap, my_svg_icon)
    
    w     - width (required)
    h     - height (optional, if not provided will be calculated)  
    txt_h - default text height
    x_ins - x insert (left and right)
    y_ins - y insert (top and bottom)
    h_gap - horizontal gap between txt strings
    '''
    svg = []
    y_so_far = y_ins
    for _tuple_ in text_tuples:
        if   type(_tuple_) is str:
            svg.append(rt.svgText(_tuple_, x_ins, y_so_far + txt_h, txt_h))
            y_so_far += txt_h + h_gap
        elif type(_tuple_) is tuple:
            my_str      = _tuple_[0]
            my_txt_h    = _tuple_[1] if len(_tuple_) >= 2 else txt_h
            my_color    = _tuple_[2] if len(_tuple_) >= 3 else None
            my_h_gap    = _tuple_[3] if len(_tuple_) >= 4 else h_gap
            my_top_gap  = _tuple_[4] if len(_tuple_) >= 5 else 0
            my_svg_icon = _tuple_[5] if len(_tuple_) >= 6 else None
            icon_w      = 0
            if my_svg_icon is not None:
                icon_w, icon_h = rt.__extractSVGWidthAndHeight__(my_svg_icon)
                svg.append(rt.__overwriteSVGXAndY__(my_svg_icon, (x_ins, my_top_gap + y_so_far + my_txt_h/2.0 - icon_h/2.0)))
                icon_w += x_ins
            svg.append(rt.svgText(my_str, x_ins + icon_w, my_top_gap + y_so_far + my_txt_h, my_txt_h, my_color))
            y_so_far += my_txt_h + my_h_gap
        else: raise Exception("Only Strings or Tuples Supported")

    my_h = h if h is not None else y_so_far + y_ins

    return f'<svg x="0" y="0" width="{w}" height="{my_h}">' + ''.join(svg) + '</svg>'

circle_svg = '<svg x="0" y="0" width="16" height="16"><circle cx="8" cy="8" r="8" fill="#ff0000"/></svg>'
rt.tile([rt.labeler(['this','is','a','test'], w=64),
         rt.labeler([('this', 24), ('is', 8, None, 0, 3), ('a', 20), ('test', 16, '#ff0000')], w=64),
         rt.labeler([('Cir', 24, None, 0, 3, circle_svg)], w=64)])


In [ ]:
_tbl_ = rt.textBlock(passage_long, word_wrap=True)
rt.tile([_tbl_.highlights(lu=entity_extractions_lu['chatgpt'])])